# Python Lab 2

現在我們要來匯出所有 Trinity Job 用到的 SQL

```{admonition} SQL 在哪裡 ， 一定難不倒你 ！

目前 Trinity Job 有可能有寫 SQL 的 元件 / Job 大概如下

1. External Command (Embedded_Script)
2. Reader Plugin
3. SQLExecutor Plugin
```

會發現如果只用 SQL 來盤點 (unnest(XPATH_XML))

必須分成 3 次 來做 ， 或重複 JOIN SUBQUERY

我們現在要一次全部到位

````{note} 
`unnest 會捨棄空集合 {} ， 所以當指定的 XPATH 不存在時 ， unnest 將不會return row`

以下兩個 SQL Return Rows 數量將不同

```sql
select DISTINCT js.stepname, dm.xmldata
, xpath('//prop[@name="SQL_SELECT_STATEMENT"]/@value', dm.xmldata::xml)::text as READER_SQL
from job j
left join jobstep js on j.jobuid = js.jobuid
left join jobstepdm dm on js.stepuid = dm.stepuid
left join jobcategory j2 on j.categoryuid = j2.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where b2.busentityname = 'FET_P'
  and j.jobname = 'STORE_PROFILE'
  and j2.categoryname = 'MART_STORE'
```

上面 SQL 有 12 筆 （ 12 個 STEP ）


```sql
select DISTINCT js.stepname, dm.xmldata
, unnest(xpath('//prop[@name="SQL_SELECT_STATEMENT"]/@value', dm.xmldata::xml))::text as READER_SQL
from job j
left join jobstep js on j.jobuid = js.jobuid
left join jobstepdm dm on js.stepuid = dm.stepuid
left join jobcategory j2 on j.categoryuid = j2.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where b2.busentityname = 'FET_P'
  and j.jobname = 'STORE_PROFILE'
  and j2.categoryname = 'MART_STORE'
```

Unnest 後 ， 會只剩下 6 筆 （ 有 Reader 的 STEP ）
````

## Lab 2

來解決上面的問題吧

首先 ， 我們改用 Python 來幫我們解析 XML 

這邊我會推薦使用 `lxml` ， 使用上比較直觀

一樣先安裝

```
pip3 install lxml
```

```
python3 -c 'import lxml'
```

接著執行 SQL

```sql
select b2.busentityname, j4.categoryname, j.jobname
, j2.stepname, j2.xmldata as STEP_XML
, j3.xmldata as DM_XML
from job j
left join jobstep j2 on j.jobuid = j2.jobuid
left join jobstepdm j3 on j2.stepuid = j3.stepuid
left join jobcategory j4 on j.categoryuid = j4.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where b2.busentityname = 'Z_ODS_BK'
ORDER BY 1,2,3,4
```

剩下的 ， 就是解析 XML & 輸出檔案了

```{admonition} 關於檔案的命名
通常我們希望用不會被用來命名 Job 、 Busentity 的字元來做分隔符號

e.g. @@ 、 || 、 %% 等

這邊我們這樣命名

`BUSENTITY`@@`CATEGORY`@@`JOB`@@`STEP`@@`SQL_TYPE`@@`SEQ`

`SQL_TYPE` 分為 `S`(Embedded_Script) 、 `E`(Executor) 、 `R`(Reader)

`SEQ` 則確保一個 STEP 有多個 Reader + Executor
```


````{admonition} lxml的簡單使用
:class: dropdown

原始XML資料均以 `TEXT` 來處理

故要先轉換成 xml object

```python
from lxml import etree

xml_object = etree.fromstring(XML)
```

之後一樣用 XPATH 來找關鍵節點

1. `Type S` : `//prop[@name="scriptsql"]`
2. `Type E` : `//prop[@name="SQL_STATEMENT"]`
3. `Type R` : `//prop[@name="SQL_SELECT_STATEMENT"]`

XPATH 將會 Return Element List 

再將每個 Element 中的 Value 取出

```python
xml_list = xml_object.xpath('//prop[@name="SQL_STATEMENT"]')

for element in xml_list:
    print(element.get('value'))
```
````

### 參考解答

````{admonition} 請參考
:class: dropdown

```python

from lxml import etree
import psycopg2


default_output_dir = 'file/SQL/'

sql = '''
select b2.busentityname, j4.categoryname, j.jobname
, j2.stepname, j2.xmldata as STEP_XML
, j3.xmldata as DM_XML
from job j
left join jobstep j2 on j.jobuid = j2.jobuid
left join jobstepdm j3 on j2.stepuid = j3.stepuid
left join jobcategory j4 on j.categoryuid = j4.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where b2.busentityname = 'Z_ODS_BK'
ORDER BY 1,2,3,4
'''

def extract_sql(stepxml, xml): 
    # Return sql list
    sql_list = []
    
    if xml:
        _xml = etree.fromstring(xml)
    else:
        _xml = etree.Element('root')  # 空資料
    
    if stepxml:
        _stepxml = etree.fromstring(stepxml)
    else:
        _stepxml = etree.Element('root')  # 空資料
    
    # Find XPATH
    scriptsql = _stepxml.xpath('//prop[@name="scriptsql"]')
    sql_path = _xml.xpath('//prop[@name="SQL_STATEMENT"]')
    reader_path = _xml.xpath('//prop[@name="SQL_SELECT_STATEMENT"]')
    
    if sql_path:    
        for sql in sql_path:
            sql_list.append(('E', sql.get('value')))
            
    if reader_path: 
        for sql in reader_path:  
            sql_list.append(('R', sql.get('value')))  
            
    if scriptsql:
        for sql in scriptsql:
            sql_list.append(('S', sql.get('value')))
            
    return sql_list

connection = psycopg2.connect(host='10.140.0.2', user='trinity', password='trinity', dbname='trinity')
cur = connection.cursor()
cur.execute(sql)

for count, rs in enumerate(cur):
    
    if count % 100 == 0:
#         print('Rows Processed : {}'.format(count))
#         print(rs)
        pass
        
    f_name = '@@'.join(rs[:4])
    
    sql_l = extract_sql(rs[4], rs[5])

    if len(sql_l) == 1:
        # Tuple 1 -> Mode  2 -> SQL
        mode = sql_l[0][0]
        _f_name = default_output_dir + f_name + '@@{}@@0.txt'.format(mode)
        
        with open(_f_name, 'w', encoding='UTF-8') as f:
                f.write(sql_l[0][1])
                
    else:
        for i, _s in enumerate(sql_l):
            mode = _s[0]
            _f_name = default_output_dir + f_name + '@@{}@@{}.txt'.format(mode, i)

            with open(_f_name, 'w', encoding='UTF-8') as f:
                f.write(_s[1])



connection.close()
```
````

## Lab 2.5

小小的延伸

前面我們透過 `lxml` 來解析 XML中的 SQL Script

可以解析 、 取出節點的資料 ， 那肯定可以 `更新` 節點的資料

接下來的我就不執行 Code 了 ， 我們來更新 `SQLExecutor Script` 後

`UPDATE` / `INSERT` 回去 ， 各位就找一隻 Job 來試試看 UPDATE 回去

但請小心 / 避免選到同一隻 Job

或是模仿 Trinity COPY Job

那下面我就講述 UPDATE 回去的部分 ， COPY Job 只陳述簡單概念 （ 其實就是更換 UUID ）

```{note} 小提醒
如果可以只用 Trinity JFDesigner 做到的事

千萬別浪費時間用 Python 寫喔 ～

e.g. 去創一個自己用的 BusinessEntity / Category / Job
```

以下用 `Z_ODS_BK` / `DATA_EVENT.SERVICE_ORDER` / `SERVICE_ORDER_DTL_COH` 做範例

所有 STEP 的所有 SQL ， 請刪除 `FOREIGN TABLE`  &  `TD6750`

````{admonition} 參考 SQL
SELECT 部分

```sql
select b2.busentityname, j4.categoryname, j.jobname
, j2.stepname, j2.xmldata as STEP_XML
, j3.xmldata as DM_XML
from job j
left join jobstep j2 on j.jobuid = j2.jobuid
left join jobstepdm j3 on j2.stepuid = j3.stepuid
left join jobcategory j4 on j.categoryuid = j4.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where b2.busentityname = 'Z_ODS_BK'
  and j4.categoryname = 'DATA_EVENT.SERVICE_ORDER'
  and j.jobname = 'SERVICE_ORDER_DTL_COH'
ORDER BY 1,2,3,4
```

UPDATE 部分

```sql
UPDATE jobstepdm
set xmldata = 
where stepuid in (
    select dm.stepuid
    from jobstepdm dm
    left join jobstep j on dm.stepuid = j.stepuid
    left join job j2 on j.jobuid = j2.jobuid
    left join jobcategory j3 on j2.categoryuid = j3.categoryuid
    left join busentitycategory b on j2.categoryuid = b.categoryuid
    left join busentity b2 on b.busentityuid = b2.busentityuid
    where b2.busentityname = 'Z_ODS_BK'
    and j3.categoryname = 'DATA_EVENT.SERVICE_ORDER'
    and j2.jobname = 'SERVICE_ORDER_DTL_COH'
    and j.stepname = 'STEP0201_TRANSFORM'
    )
```
````

### 參考解答

````{admonition} 請參考，這題比較深
:class: dropdown

```python
import psycopg2
import pandas as pd
import re


sql = """
select b2.busentityname, j4.categoryname, j.jobname
, j2.stepname, j2.xmldata as STEP_XML
, j3.xmldata as DM_XML
from job j
left join jobstep j2 on j.jobuid = j2.jobuid
left join jobstepdm j3 on j2.stepuid = j3.stepuid
left join jobcategory j4 on j.categoryuid = j4.categoryuid
left join busentitycategory b on j.categoryuid = b.categoryuid
left join busentity b2 on b.busentityuid = b2.busentityuid
where b2.busentityname = 'Z_ODS_BK'
  and j4.categoryname = 'DATA_EVENT.SERVICE_ORDER'
  and j.jobname = 'SERVICE_ORDER_DTL_COH'
  and j2.stepname = 'STEP0201_TRANSFORM'
ORDER BY 1,2,3,4
"""

update_sql = """
UPDATE jobstepdm
set xmldata = %s
where stepuid in (
    select dm.stepuid
    from jobstepdm dm
    left join jobstep j on dm.stepuid = j.stepuid
    left join job j2 on j.jobuid = j2.jobuid
    left join jobcategory j3 on j2.categoryuid = j3.categoryuid
    left join busentitycategory b on j2.categoryuid = b.categoryuid
    left join busentity b2 on b.busentityuid = b2.busentityuid
    where b2.busentityname = '{busentity}'
    and j3.categoryname = '{category}'
    and j2.jobname = '{job}'
    and j.stepname = '{step}'
    )
"""
xml_comment = '<!-- Data Management XML Definition, generated by JFDesigner V4.1.0.169 build 20180920 -->\n{}'
connection = psycopg2.connect(host='10.140.0.2', user='trinity', password='trinity', dbname='trinity')
cur = connection.cursor()
cur.execute(sql)

rs_li = cur.fetchall()

for rs in rs_li:
    
    where_clause = {
        'busentity': rs[0],
        'category': rs[1],
        'job': rs[2],
        'step': rs[3],
    }
    tree = etree.fromstring(rs[5])
    for i, _sql in enumerate(tree.xpath('//prop[@name="SQL_STATEMENT"]/@value')):
        _sql = re.sub('FOREIGN TABLE', '', _sql, flags=re.I)
        _sql = re.sub('@TD6750', '', _sql, flags=re.I)
        
        # 更新 xml element
        tree.xpath('//prop[@name="SQL_STATEMENT"]')[i].attrib['value'] = _sql
    
    _update_sql = update_sql.format(**where_clause)
    
    text = xml_comment.format(etree.tostring(tree).decode('UTF-8').replace("'", '&apos;'))
    
    cur.execute(_update_sql, (text,))

connection.commit()
```
````